# 平均値の差の検定

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

sns.set()
%precision 3
%matplotlib inline
warnings.filterwarnings('ignore')

## 2変量データに対するt検定

* 2つの変数の間で、平均値に差があるかどうかを判断する
* 例: 薬を飲む前と飲んだ後で、体温に差が出るかどうかを調べる場合など、「同じ対象を異なった条件で2回測定して、その違いを見る」といった場合に対応のあるt検定を用いる
  * 飲む前と飲んだ後の差分の平均値が0と異なれば、「薬を飲む前と飲んだ後で体温が異なる」ということを主張することができる(有意水準=5%とする)


In [2]:
paired_test_data = pd.read_csv('../data/3-9-1-paired-t-test.csv')
paired_test_data

,person,medicine,body_temperature
0,A,before,36.2
1,B,before,36.2
2,C,before,35.3
3,D,before,36.1
4,E,before,36.1
5,A,after,36.8
6,B,after,36.1
7,C,after,36.8
8,D,after,37.1
9,E,after,36.9


## 対応のあるt検定

薬を飲む前と飲んだ後における、体温の差を計算

In [11]:
# 薬を飲む前と飲んだ後の標本平均
before = paired_test_data.query('medicine == "before"')['body_temperature']
after = paired_test_data.query('medicine == "after"')['body_temperature']

before = np.array(before)
after = np.array(after)

diff = after - before
print(diff)
# t検定 比較対象値=0(差が0であるかを検証するため)
# 有意水準: 5%= pvalue<=0.05
print(stats.ttest_1samp(diff, 0))
print(stats.ttest_rel(after, before))
# よって、pvalue<0.05であるため、「薬を飲む前と後の体温は異なる」と主張可能になる

[ 0.6 -0.1  1.5  1.   0.8]
Ttest_1sampResult(statistic=2.901693483620596, pvalue=0.044043109730074276)
Ttest_relResult(statistic=2.901693483620596, pvalue=0.044043109730074276)


## 対応の無いt検定

- 対応の無いt検定は「平均値の差」に注目する
- 対応のあるt検定では「データの差」をとってから1群のt検定を実施した点と異なる

$$
t値 = \frac{標本平均-比較対象}{標本偏差/\sqrt{サンプルサイズ}} = \frac{標本平均-比較対象}{標準誤差}
$$

変数xとyの平均値の差を検定するとする。変数xは「大きい針で釣った魚の体長」で変数yは「小さい針で釣った魚の体長」とする。対応の無いt検定のt値は以下のような数式になる

$$ t = \frac{\hat{\mu}_x-\hat{\mu}_y}{\sqrt{\hat{\sigma^2}_x/m+\hat{\sigma^2}_y/m}} $$

* `\hat{\mu}_x`はxの標本平均で、`\hat{\mu}_y`はyの標本平均
* mはxのサンプルサイズで、nはyのサンプルサイズ
* `\hat{\sigma^2}_x`はxの不偏分散で、`\hat{\sigma^2}_y`はyの不偏分散

基本的には、1群のt検定におけるt値と同様だが、2つの変数で分散が異なっていることを仮定した上で計算をしている。また、2群の場合は、t分布の自由度が複雑になる。`Welch`近似法と呼ばれる方法を使ってp値を計算することになる。

In [13]:
mean_bef = sp.mean(before)
mean_aft = sp.mean(after)

sigma_bef = sp.var(before, ddof=1)
sigma_aft = sp.var(after, ddof=1)

m = len(before)
n = len(after)
# t値
t_value = (mean_bef-mean_aft) / sp.sqrt((sigma_bef/m + sigma_aft/n))
print(f't_value: {t_value}')
# equal_var=False は分散が異なることを仮定したt検定(=Welch)
print(stats.ttest_ind(after, before, equal_var=False))

t_value: -3.1557282344421034
Ttest_indResult(statistic=3.1557282344421034, pvalue=0.013484775682079892)


p値が「対応のあるt検定」の結果と異なっている。
同じデータに対して同じ目的の検定を行っても、検定手法が変わるとp値も変わる

- 古い統計入門書では、「データの等分散性を検定した後、分散が異なることを仮定したt検定か、分散が等しいことを仮定したt検定を使いわkる」と書かれていた
- しかし、当分参加どうか調べるまでもなく、常に分散が異なることを仮定したt検定を使っても問題はない。つまり、常にWelchの検定を使って良い

## pハッキング

- 平均値の差の検定という、1つの目的のためにも、複数の検定手法がある
  - Welch
  - マンホイットニーのU検定
- 悠一成の出やすさは各々の検定手法により異なる
- 対応のあるt検定と対応の無いt検定では、p値は異なる
  - 有意差が得られるまで、何度も何度も検定の手法を切り替えていく人がいる
  - しかし、そのようにして有意差が得られたとして、何の意味があるか。
  - 有意差とは、サンプルサイズやデータのばらつきなど特徴を包括的にまとめて意味があるかを判断する考え方である

p値という数値にこだわって「自分の欲しい結果を得るための分析」にしてしまうことは最も避けないといけない
*分析者は、他の対象にはどうであれ、少なくとも1つ、データにだけは誠実でいなければならない*

